<a href="https://colab.research.google.com/github/ddavila9789/Machine-Learning-with-Apache-Spark-Final-Project/blob/main/Final_Project_(Airfoil_Noise_Prediction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

## Part 1 - Perform ETL activity


### Task 1 - Import required libraries


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

### Task 2 - Create a spark session


In [ ]:
#Create a SparkSession

spark = SparkSession.builder.appName("Final Project").getOrCreate()

### Task 3 - Load the csv file into a dataframe


Download the data file.

NOTE : Please ensure you use the dataset below and not the original dataset mentioned above.


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv


--2024-02-19 03:37:25--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv’

NASA_airfoil_noise_ 100%[===================>]  59.26K  --.-KB/s    in 0.05s   

2024-02-19 03:37:26 (1.19 MB/s) - ‘NASA_airfoil_noise_raw.csv’ saved [60682/60682]



Load the dataset into the spark dataframe


In [ ]:
# Load the dataset that you have downloaded in the previous task
df = spark.read.csv("NASA_airfoil_noise_raw.csv", header=True, inferSchema=True)

### Task 4 - Print top 5 rows of the dataset


In [ ]:
df.show(5)

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
+---------+-------------+-----------+------------------+-----------------------+----------+
only showing top 5 rows



### Task 6 - Print the total number of rows in the dataset


In [ ]:
rowcount1 = df.count()
print(rowcount1)

1522


### Task 7 - Drop all the duplicate rows from the dataset


In [ ]:
df = df.dropDuplicates()

### Task 8 - Print the total number of rows in the dataset


In [ ]:
rowcount2 = df.count()
print(rowcount2)


1503


### Task 9 - Drop all the rows that contain null values from the dataset


In [ ]:
df = df.dropna()

### Task 10 - Print the total number of rows in the dataset


In [ ]:
rowcount3 = df.count()
print(rowcount3)


1499


### Task 11 - Rename the column "SoundLevel" to "SoundLevelDecibels"Drop


In [ ]:
df = df.withColumnRenamed("SoundLevel","SoundLevelDecibels")

### Task 12 - Save the dataframe in parquet formant, name the file as "NASA_airfoil_noise_cleaned.parquet"


In [ ]:
df.write.parquet("NASA_airfoil_noise_cleaned.parquet")

#### Part 1 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [ ]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


## Part - 2 Create a  Machine Learning Pipeline


### Task 1 - Load data from "NASA_airfoil_noise_cleaned.parquet" into a dataframe


In [ ]:
df = spark.read.parquet("NASA_airfoil_noise_cleaned.parquet")

### Task 2 - Print the total number of rows in the dataset


In [ ]:
rowcount4 = df.count()
print(rowcount4)

1499


### Task 3 - Define the VectorAssembler pipeline stage


In [ ]:
df.show(1)

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|      630|          0.0|     0.3048|              31.7|             0.00331266|           129.095|
+---------+-------------+-----------+------------------+-----------------------+------------------+
only showing top 1 row



Stage 1 - Assemble the input columns into a single column "features". Use all the columns except SoundLevelDecibels as input features.


In [ ]:
assembler = VectorAssembler(inputCols=['Frequency','AngleOfAttack','ChordLength','FreeStreamVelocity','SuctionSideDisplacement'], outputCol="features")

### Task 4 - Define the StandardScaler pipeline stage


Stage 2 - Scale the "features" using standard scaler and store in "scaledFeatures" column


In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

### Task 5 - Define the StandardScaler pipeline stage


Stage 3 - Create a LinearRegression stage to predict "SoundLevelDecibels"


In [ ]:
lr = LinearRegression(featuresCol="features", labelCol="SoundLevelDecibels")

### Task 6 - Build the pipeline


Build a pipeline using the above three stages


In [ ]:
pipeline = Pipeline(stages=[assembler, scaler, lr])

### Task 7 - Split the data


In [ ]:
# Split the data into training and testing sets with 70:30 split.
# set the value of seed to 42
# the above step is very important. DO NOT set the value of seed to any other value other than 42.

(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)

### Task 8 - Fit the pipeline


In [ ]:
# Fit the pipeline using the training data
pipelineModel = pipeline.fit(trainingData)

#### Part 2 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [ ]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


## Part 3 - Evaluate the Model


### Task 1 - Predict using the model


In [ ]:
# Make predictions on testing data
predictions = pipelineModel.transform(testingData)

### Task 2 - Print the MSE


In [ ]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mse")
mse = evaluator.evaluate(predictions)
print(mse)

25.80619595546937


### Task 3 - Print the MAE


In [ ]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

3.969683112411967


### Task 4 - Print the R-Squared(R2)


In [ ]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)

0.5004963012661714


#### Part 3 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [ ]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  25.81
Mean Absolute Error =  3.97
R Squared =  0.5
Intercept =  132.27


## Part 4 - Persist the Model


### Task 1 - Save the model to the path "Final_Project"


In [ ]:
pipelineModel.write().save("Final_Project")

### Task 2 - Load the model from the path "Final_Project"


In [ ]:
# Load the pipeline model you have created in the previous step
loadedPipelineModel = PipelineModel.load("Final_Project")

### Task 3 - Make predictions using the loaded model on the testdata


In [ ]:
# Use the loaded pipeline model and make predictions using testingData
predictions = loadedPipelineModel.transform(testingData)

### Task 4 - Show the predictions


In [ ]:
#show top 5 rows from the predections dataframe. Display only the label column and predictions
predictions.select("SoundLevelDecibels","prediction").show(5)

+------------------+------------------+
|SoundLevelDecibels|        prediction|
+------------------+------------------+
|           128.545|121.11789021572748|
|           130.898|122.61643555722858|
|           109.951| 127.5646516314007|
|           112.506|129.23201861714693|
|           130.089| 122.0386332423671|
+------------------+------------------+
only showing top 5 rows



#### Part 4 - Evaluation



Run the code cell below.<br>
Use the answers here to answer the final evaluation quiz in the next section.<br>
If the code throws up any errors, go back and review the code you have written.


In [ ]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[0].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  3
Coefficient for Frequency is -0.0012
Coefficient for AngleOfAttack is -0.3691
Coefficient for ChordLength is -36.116
Coefficient for FreeStreamVelocity is 0.1007
Coefficient for SuctionSideDisplacement is -163.9253


### Stop Spark Session


In [ ]:
spark.stop()